# Parsing HTML, Beautiful Soup, Mongo DB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

- Наименование вакансии.
- Предлагаемую зарплату (отдельно минимальную и максимальную).
- Ссылку на саму вакансию.
- Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from pymongo import MongoClient
from pprint import pprint

In [2]:
client = MongoClient('127.0.0.1', 27017)

DataBase = client['hhParsing']

collection = DataBase['questionnaires']

In [3]:
def _parser_item_hh(item):
    vacancy_dict = {}
    salary_min = None
    salary_max = None
    salary_currency = None
    # vacancy_name
    vacancy_name = item.find('span', {'class': 'resume-search-item__name'}) \
                    .getText() \
                    .replace(u'\xa0', u' ')   
    vacancy_dict['vacancy_name'] = vacancy_name    
    # company_name
    company_name = item.find('div', {'class': 'vacancy-serp-item__meta-info'}) \
                    .find('a') \
                    .getText() 
    vacancy_dict['company_name'] = company_name
    # city
    city = item.find('span', {'class': 'vacancy-serp-item__meta-info'}) \
                .getText() \
                .split(', ')[0]
    vacancy_dict['city'] = city
    #metro station
    metro_station = item.find('span', {'class': 'vacancy-serp-item__meta-info'}).findChild()
    if not metro_station:
        metro_station = None
    else:
        metro_station = metro_station.getText()
    vacancy_dict['metro_station'] = metro_station
    #salary
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})        
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        salary_1  =re.findall(r"[0-9]+[ ][0-9]+", salary)
        salary_1 = [x.replace(' ', '') for x in salary_1]
        salary = re.split(r'\s|-', salary)
        if salary[0] == 'до':
            salary_max = int(salary_1[0])
            salary_min = None
        elif salary[0] == 'от':
            salary_min = int(salary_1[0])
            salary_max = None
        elif len(salary_1)>1:
            salary_min = int(salary_1[0])
            salary_max = int(salary_1[1])
        salary_currency = salary[len(salary)-1] 
    vacancy_dict['salary_min'] = salary_min
    vacancy_dict['salary_max'] = salary_max
    vacancy_dict['salary_currency'] = salary_currency
    # link
    vacancy_link = item.find('a', {'data-qa': 'vacancy-serp__vacancy-title'})['href']
    vacancy_dict['vacancy_link'] = vacancy_link 
    # site
    vacancy_dict['site'] = 'hh.ru'
    return vacancy_dict

In [4]:
def _parser_hh(vacancy):
    
    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'bloko-button'})[-2].getText())
            
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(_parser_item_hh(item))
                vacancy = _parser_item_hh(item)
                
                if _is_exists('vacancy_link', vacancy['vacancy_link']):
                    collection.update_one({'vacancy_link': vacancy['vacancy_link']}, {'$set': vacancy})
                else:
                    collection.insert_one(vacancy)
                
    return vacancy_date

In [5]:
def parser_vacancy(vacancy):
        
    vacancy_list = []
    vacancy_list.extend(_parser_hh(vacancy))
    
    df = pd.DataFrame(vacancy_list)

    return df

- Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [6]:
def _is_exists(name_tags, field):
        return bool(collection.find_one({name_tags: { "$in": [field]}}))

In [7]:
vacancy = 'Python'
df = parser_vacancy(vacancy)

- Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД.

In [10]:
i = 0 
for item in collection.find({}):
    pprint(item)
    i += 1
    if i == 5:
        break

{'_id': ObjectId('61225d275a608d1964c0a336'),
 'city': 'Москва',
 'company_name': 'Сбер. IT',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/47084105?from=vacancy_search_list&query=Python',
 'vacancy_name': 'Python-разработчик'}
{'_id': ObjectId('61225d275a608d1964c0a337'),
 'city': 'Киев',
 'company_name': 'ООО\xa0Чисельні Технології',
 'metro_station': None,
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/47174921?from=vacancy_search_list&query=Python',
 'vacancy_name': 'Math + Python developer'}
{'_id': ObjectId('61225d275a608d1964c0a338'),
 'city': 'Москва',
 'company_name': 'КА Натальи Зотовой',
 'metro_station': None,
 'salary_currency': 'USD',
 'salary_max': 5500,
 'salary_min': None,
 'site': 'hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/47139730?from=vacancy_search_list&query=Python',
 'vacancy_n

2. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [11]:
def print_salary(salary):
        objects = collection.find({'salary_max': {'$gt': salary}})
        for obj in objects:
            pprint(obj)

In [12]:
print_salary(300_000)

{'_id': ObjectId('61225d275a608d1964c0a33f'),
 'city': 'Санкт-Петербург',
 'company_name': 'ООО\xa0Вайт Код',
 'metro_station': 'Петроградская',
 'salary_currency': 'руб.',
 'salary_max': 400000,
 'salary_min': 300000,
 'site': 'hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/47262469?from=vacancy_search_list&query=Python',
 'vacancy_name': 'Python developer (Senior)'}
{'_id': ObjectId('61225fc35a608d1964c0a77b'),
 'city': 'Москва',
 'company_name': 'CATAPULTO.RU',
 'metro_station': 'Павелецкая',
 'salary_currency': 'руб.',
 'salary_max': 400000,
 'salary_min': 300000,
 'site': 'hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/44433173?from=vacancy_search_list&query=Python',
 'vacancy_name': 'Team Lead Python/Django'}
{'_id': ObjectId('61225fc65a608d1964c0a7c3'),
 'city': 'Москва',
 'company_name': 'Global Hunter',
 'metro_station': None,
 'salary_currency': 'руб.',
 'salary_max': 350000,
 'salary_min': None,
 'site': 'hh.ru',
 'vacancy_link': 'https://hh.ru/vacancy/46641470?from=vacanc